**извлечение признаков из текста на естественном языке**

очистка текста и токенизация

_Евгений Борисов <esborisov@sevsu.ru>_

## библиотеки

In [1]:
import re
# import numpy as np
# import numpy.random as rng
import pandas as pd
# from tqdm import tqdm
from tqdm.notebook import tqdm

# np.set_printoptions(precision=2) # вывод на печать чисел до 2 знака
pd.options.display.max_colwidth = 200 

tqdm.pandas()

---

In [2]:
# https://habr.com/ru/post/516098/
# Corus — коллекция русскоязычных NLP-датасетов
# https://github.com/yutkin/Lenta.Ru-News-Dataset/

## тексты

In [4]:
# загружаем тексты
data = pd.read_pickle('../data/text/news.pkl.gz')
print('записей:',len(data))
data.sample(5)

записей: 3196


,text,tag
2056,Хавбек Челси Сеск Фабрегас отказался от финансово очень заманчивого предложения из Китая и желает остаться в Англии.\n\nХавбек Челси Сеск Фабрегас отказался от финансово очень заманчивого предложе...,sport
539,Под Петриковом водитель не заметил припаркованный на обочине Opel: пострадали три человека\n\n5 декабря 2016 в 17:19\n\nAUTO.TUT.BY\n\nДТП произошло 4 декабря в Петриковском районе на трассе Птичь...,incident
385,"Highway Михаила Спирита на радио ""Мегаполис FM"" Эфирная сетка радиостанции\n""Мегаполис FM"" обогатилась на еще одно шоу. 28 ноября вслед за программой\nDj Паши Крейца ""Танцпол"" из приемников зазвуч...",culture
2671,"Вашингтон, 11 декабря. Американские ученые выяснили, что происходит с мозгом при попытке решить сразу несколько задач одновременно. Оказалось, что он попросту начинает «раскалываться»: одна часть ...",science
967,Лукашенко: падение товарооборота в ЕАЭС негативно сказывается на имидже интеграционного объединения\n\n6 декабря 2016 в 11:26\n\nTUT.BY\n\nПадение товарооборота в ЕАЭС негативно сказывается на ими...,economics


In [5]:
# длина строк
pd.DataFrame(data['text'].str.len()).describe([.1,.25,.5,.75,.95]).astype(int).T

,count,mean,std,min,10%,25%,50%,75%,95%,max
text,3196,1720,2177,25,370,675,1070,1813,5761,30710


In [6]:
# количество категорий
data['tag'].drop_duplicates().count()

13

In [7]:
# собираем словарь из текстов
def get_vocabulary(ds):
    vcb = [ set(s) for s in ds.tolist() ]
    return sorted(set.union(*vcb))

## очистка и токенизация 

In [8]:
# применяет список замен pat к строке s
def replace_patterns(s,pat):
    if len(pat)<1: return s
    return  replace_patterns( re.sub(pat[0][0],pat[0][1],s), pat[1:] )

# нормализация текста
def string_normalizer(s):
    pat = [
       [r'ё','е'] # замена ё для унификации
       ,[r'</?[a-z]+>',' '] # удаляем xml
       ,[r'[^a-zа-я\- ]+',' '] # оставляем только буквы, пробел и -
       ,[r' -\w+',' '] # удаляем '-й','-тый' и т.п.
       ,[r'\w+- ',' ']
       ,[r' +',' '] # удаляем повторы пробелов
    ]
    return replace_patterns(s.lower(),pat).strip()

data['ctext'] = data['text'].progress_apply(string_normalizer)

  0%|          | 0/3196 [00:00<?, ?it/s]

In [9]:
# разрезаем стоки на слова
def tokenize(s): 
    return [ w for w in s.split(' ') if (len(w)>1) ]

data['ctext'] = data['ctext'].progress_apply( tokenize )

  0%|          | 0/3196 [00:00<?, ?it/s]

In [10]:
vcb =  get_vocabulary( data['ctext'] )
print('словарь %i слов'%(len(vcb)))
# pd.DataFrame( vcb ).to_csv('voc0.txt',index=False,header=False)

словарь 83094 слов


In [11]:
data.sample(3)

,text,tag,ctext
2869,"Новый кроссовер приедет в Россию в двух кузовных модификациях: стандартной и длиннобазной XLV, которая будет длиннее на 230мм. Для обех версий в России предусмотрен только один 1,6-литровый бензин...",auto,"[новый, кроссовер, приедет, россию, двух, кузовных, модификациях, стандартной, длиннобазной, xlv, которая, будет, длиннее, на, мм, для, обех, версий, россии, предусмотрен, только, один, бензиновый..."
3101,"Мурманские оленеводы отдыхают в санатории В санаторно-гостиничный комплекс\n""Изовела"" прибыла группа работников оленеводческих хозяйств из числа\nпредставителей коренных малочисленных народов Севе...",social,"[мурманские, оленеводы, отдыхают, санатории, санаторно-гостиничный, комплекс, изовела, прибыла, группа, работников, оленеводческих, хозяйств, из, числа, представителей, коренных, малочисленных, на..."
983,"Лукашенко примет польского сенатора, которого 4 года назад не пустили в Беларусь\n\n3 декабря 2016 в 17:13\n\nTUT.BY\n\nПредседатель польского сената Станислав Карчевски во главе делегации посетит...",politics,"[лукашенко, примет, польского, сенатора, которого, года, назад, не, пустили, беларусь, декабря, tut, by, председатель, польского, сената, станислав, карчевски, во, главе, делегации, посетит, декаб..."


## стеминг

In [12]:
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords as nltk_stopwords
# from nltk import download as nltk_download
# nltk_download('stopwords')

stopwords = set(nltk_stopwords.words('russian'))
stemmer = SnowballStemmer('russian')

# выкидываем stopwords, выполняем стеминг
def stem(s): 
    return [ stemmer.stem(w) for w in s if w not in stopwords ]

In [13]:
data['ctext'] = data['ctext'].progress_apply( stem )

  0%|          | 0/3196 [00:00<?, ?it/s]

In [14]:
vcb =  get_vocabulary( data['ctext'] )
print('словарь %i слов'%(len(vcb)))
# pd.DataFrame( vcb ).to_csv('voc1.txt',index=False,header=False)

словарь 36381 слов


In [15]:
data.sample(3)

,text,tag,ctext
1368,"Ученые нашли различие в зрении мужчин и женщин\n\n30 ноября 2016 в 10:32\n\n42.TUT.BY\n\nЖенщины и мужчины смотрят на лица и интерпретируют визуальную информацию по-разному, что может приводить к ...",tech,"[учен, нашл, различ, зрен, мужчин, женщин, ноябр, tut, by, женщин, мужчин, смотр, лиц, интерпретир, визуальн, информац, по-разн, привод, гендерн, различ, пониман, визуальн, информац, научн, работ,..."
3182,Маяк (34374.ru) Социальные пенсии подросли В связи с принятием постановления\nПравительства Российской Федерации от 21 июня 2010 г. N457 ?Об утверждении\nкоэффициента дополнительной индексации с 1...,social,"[маяк, ru, социальн, пенс, подросл, связ, принят, постановлен, правительств, российск, федерац, июн, утвержден, коэффициент, дополнительн, индексац, июл, социальн, пенс, июл, год, социальн, пенс, ..."
3127,"Восток-Медиа (vostokmedia.com) Дальневосточный фестиваль художественных\nремесел ""Живая нить времен"" пройдет в Хабаровске. В нем примут участие\nмастера художественных промыслов и декоративно-прик...",culture,"[восток-мед, vostokmedia, com, дальневосточн, фестивал, художествен, ремесел, жив, нит, врем, пройдет, хабаровск, нем, примут, участ, мастер, художествен, промысл, декоративно-прикладн, искусств, ..."


## токенайзеры nltk

In [21]:
import re
import gzip

# загружаем текст ...
with gzip.open('../data/text/dostoevsky-besy-p2.txt.gz','rt') as f:  
    text = f.read()[105:] # ...и выкидываем заголовок

print('символов:%i\n'%(len(text)))
print(text[:364].strip())

символов:465490

Теперь, когда уже все прошло, и я пишу хронику, мы уже знаем в чем дело; но тогда мы еще ничего не знали, и естественно, что нам представлялись странными разные вещи. По крайней мере мы со Степаном Трофимовичем в первое время заперлись и с испугом наблюдали издали. Я-то кой-куда еще выходил и по-прежнему приносил ему разные вести, без чего он и пробыть не мог.


In [22]:
from random import sample

from nltk.tokenize import sent_tokenize as nltk_sentence_split
from nltk.tokenize import word_tokenize as nltk_tokenize_word

text = [ 
    nltk_tokenize_word(s) # разбиваем предложения на слова
    for s in tqdm( nltk_sentence_split(text) ) # режем текст на отдельные предложения
]

print('предложений: %i\n'%(len(text)))

sample(text,5)

  0%|          | 0/5556 [00:00<?, ?it/s]

предложений: 5556



[['-',
  'нетерпеливо',
  'окликнул',
  'он',
  'Кириллова',
  'уже',
  'неподалеку',
  'от',
  'дому',
  '.'],
 ['``', 'Да', 'что', 'же', 'лошади', '?', "''"],
 ['-', 'Да', 'вы', 'это', 'нарочно', ',', 'что', 'ли', '?'],
 ['-', 'Да-с', ',', 'ясно', ',', 'но', 'кого', 'же', 'он', 'просит', '?'],
 ['Я',
  'не',
  'аллегорию',
  ',',
  'я',
  'просто',
  'лист',
  ',',
  'один',
  'лист',
  '.']]